### TMBD Data Extraction - TMDB API

#### 1. Import the necessary libraries

In [1]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import time

#### 2. Import the links dataset from which we are going to extract the data

In [2]:
IDs = pd.read_csv('links.csv')
IDs = IDs[['movieId', 'tmdbId']]

In [3]:
IDs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 152.3 KB


In [4]:
IDs[IDs.tmdbId.isna()]

,movieId,tmdbId
624,791,NaN
843,1107,NaN
2141,2851,NaN
3027,4051,NaN
5532,26587,NaN
5854,32600,NaN
6059,40697,NaN
7382,79299,NaN


There are 7 movies without TMDb ID, so they are going to be removed

In [5]:
IDs_notnull = IDs[IDs.tmdbId.isna() == False]
IDs_notnull['tmdbId'] = IDs_notnull['tmdbId'].astype(np.int64)
dict_IDs = dict(zip(IDs_notnull.movieId, IDs_notnull.tmdbId))

#### 3. TMDb movies data extraction

In [6]:
# API key obtained after registration on the TMDb platform
api_key = 'fd69883de355b2d1886ce73b333e81c5'
host_url = 'https://api.themoviedb.org/3/movie/'

# Initialize the dataframe
columns = ['tmdbID', 'movieID', 'adult', 'budget', 'original_language', 'popularity', 'poster_path', 
           'production_companies', 'production_countries', 'revenue', 'runtime', 
           'vote_average', 'vote_count']
df_TMDB = pd.DataFrame(columns = columns)


# TMDB Data Extraction
print('TMDB Data Extraction')
print('--------------------')
start_time = time.time()
i = 0
for movieID, tmdbID in dict_IDs.items():
    # Control text to check if the movie data was successfully extracted
    i += 1
    print(str(i) + ' out of', str(len(IDs_notnull)) + ': movieID = ' + str(movieID) + ', ' + 'tmdbID = ' + str(tmdbID))
    # Request to the TMDb url through an API key
    url = host_url + str(tmdbID) + '?api_key=' + api_key
    data_json = requests.get(url).json()
    # Normalization of the json file to convert it to a pandas dataframe
    dataset_tmp = pd.json_normalize(data_json)

    columns = ['adult', 'budget', 'original_language', 'popularity', 'poster_path', 
               'production_companies', 'production_countries', 'revenue', 'runtime', 
               'vote_average', 'vote_count']
    try:
        dataset_tmp = dataset_tmp[columns]

        # IDs
        dataset_tmp['tmdbID'] = tmdbID
        dataset_tmp['movieID'] = movieID

        # Companies
        try:
            companies = dataset_tmp.production_companies[0]
            companies_list = [company['name'] for company in companies]
            all_companies = ' | '.join(companies_list)
            if all_companies == "":
                dataset_tmp['production_companies'] = "None"
                print(' -> production_companies = None')
            else:
                dataset_tmp['production_companies'] = all_companies
        except:
            print(' -> production_companies = Error')
            dataset_tmp['production_companies'] = "None"

        # Countries
        try:
            countries = dataset_tmp.production_countries[0]
            countries_list = [country['name'] for country in countries]
            all_countries = ' | '.join(countries_list)
            
            if all_countries == "":
                dataset_tmp['production_countries'] = "None"
                print(' -> production_countries = None')
            else:
                dataset_tmp['production_countries'] = all_countries
        except:
            print(' -> production_countries = Error')
            dataset_tmp['production_countries'] = "None"
        
        # Poster image
        try:
            dataset_tmp['poster_path'] = 'http://image.tmdb.org/t/p/original' + dataset_tmp.poster_path[0]
        except:
            print(' -> poster_path = None')
            dataset_tmp['poster_path'] = "None"

        # Append the movie to the dataframe
        df_TMDB = df_TMDB.append(dataset_tmp)

        columns = ['tmdbID', 'movieID', 'adult', 'budget', 'original_language', 'popularity', 
                   'poster_path', 'production_companies', 'production_countries', 'revenue', 'runtime', 
                   'vote_average', 'vote_count']
        df_TMDB = df_TMDB[columns]
    except:
        print(' -> ERROR')
        continue
# Calculate how much time did the execution take
end_time = time.time()    
total_time = round((end_time - start_time)/60, 2)
print('-------------------------------')
print('TMDB Data Extraction Completed!')
print('Elapsed Time: ', str(total_time), ' minutes')

TMDB Data Extraction
--------------------
1 out of 9734: movieID = 1, tmdbID = 862
2 out of 9734: movieID = 2, tmdbID = 8844
3 out of 9734: movieID = 3, tmdbID = 15602
4 out of 9734: movieID = 4, tmdbID = 31357
5 out of 9734: movieID = 5, tmdbID = 11862
6 out of 9734: movieID = 6, tmdbID = 949
7 out of 9734: movieID = 7, tmdbID = 11860
8 out of 9734: movieID = 8, tmdbID = 45325
9 out of 9734: movieID = 9, tmdbID = 9091
10 out of 9734: movieID = 10, tmdbID = 710
11 out of 9734: movieID = 11, tmdbID = 9087
12 out of 9734: movieID = 12, tmdbID = 12110
13 out of 9734: movieID = 13, tmdbID = 21032
14 out of 9734: movieID = 14, tmdbID = 10858
15 out of 9734: movieID = 15, tmdbID = 1408
16 out of 9734: movieID = 16, tmdbID = 524
17 out of 9734: movieID = 17, tmdbID = 4584
18 out of 9734: movieID = 18, tmdbID = 5
19 out of 9734: movieID = 19, tmdbID = 9273
20 out of 9734: movieID = 20, tmdbID = 11517
21 out of 9734: movieID = 21, tmdbID = 8012
22 out of 9734: movieID = 22, tmdbID = 1710
23 out

173 out of 9734: movieID = 204, tmdbID = 3512
174 out of 9734: movieID = 205, tmdbID = 52856
175 out of 9734: movieID = 206, tmdbID = 77350
176 out of 9734: movieID = 207, tmdbID = 9560
177 out of 9734: movieID = 208, tmdbID = 9804
178 out of 9734: movieID = 209, tmdbID = 31611
179 out of 9734: movieID = 210, tmdbID = 65796
 -> production_countries = None
180 out of 9734: movieID = 211, tmdbID = 49805
181 out of 9734: movieID = 212, tmdbID = 26258
182 out of 9734: movieID = 213, tmdbID = 50797
183 out of 9734: movieID = 214, tmdbID = 19155
184 out of 9734: movieID = 215, tmdbID = 76
185 out of 9734: movieID = 216, tmdbID = 11017
186 out of 9734: movieID = 217, tmdbID = 37141
187 out of 9734: movieID = 218, tmdbID = 9382
188 out of 9734: movieID = 219, tmdbID = 6715
189 out of 9734: movieID = 220, tmdbID = 18256
190 out of 9734: movieID = 222, tmdbID = 22625
191 out of 9734: movieID = 223, tmdbID = 2292
192 out of 9734: movieID = 224, tmdbID = 1909
193 out of 9734: movieID = 225, tmdbID

346 out of 9734: movieID = 388, tmdbID = 39953
347 out of 9734: movieID = 389, tmdbID = 41580
 -> production_companies = None
348 out of 9734: movieID = 390, tmdbID = 315
349 out of 9734: movieID = 391, tmdbID = 22067
350 out of 9734: movieID = 393, tmdbID = 11667
351 out of 9734: movieID = 405, tmdbID = 8011
352 out of 9734: movieID = 406, tmdbID = 171857
 -> production_companies = None
 -> production_countries = None
 -> poster_path = None
353 out of 9734: movieID = 407, tmdbID = 2654
354 out of 9734: movieID = 408, tmdbID = 18069
355 out of 9734: movieID = 409, tmdbID = 19092
356 out of 9734: movieID = 410, tmdbID = 2758
357 out of 9734: movieID = 412, tmdbID = 10436
358 out of 9734: movieID = 413, tmdbID = 13595
359 out of 9734: movieID = 414, tmdbID = 46094
360 out of 9734: movieID = 415, tmdbID = 9409
361 out of 9734: movieID = 416, tmdbID = 11853
362 out of 9734: movieID = 417, tmdbID = 16771
363 out of 9734: movieID = 418, tmdbID = 46924
364 out of 9734: movieID = 419, tmdbID =

518 out of 9734: movieID = 602, tmdbID = 124472
 -> production_countries = None
519 out of 9734: movieID = 605, tmdbID = 7300
520 out of 9734: movieID = 606, tmdbID = 10824
521 out of 9734: movieID = 608, tmdbID = 275
522 out of 9734: movieID = 609, tmdbID = 25059
523 out of 9734: movieID = 610, tmdbID = 11827
524 out of 9734: movieID = 611, tmdbID = 8766
525 out of 9734: movieID = 612, tmdbID = 23570
526 out of 9734: movieID = 613, tmdbID = 47333
527 out of 9734: movieID = 615, tmdbID = 51242
528 out of 9734: movieID = 616, tmdbID = 10112
529 out of 9734: movieID = 617, tmdbID = 4307
530 out of 9734: movieID = 618, tmdbID = 46029
531 out of 9734: movieID = 619, tmdbID = 32308
532 out of 9734: movieID = 626, tmdbID = 28121
533 out of 9734: movieID = 627, tmdbID = 12520
534 out of 9734: movieID = 628, tmdbID = 1592
535 out of 9734: movieID = 631, tmdbID = 19042
536 out of 9734: movieID = 632, tmdbID = 38884
537 out of 9734: movieID = 633, tmdbID = 47449
538 out of 9734: movieID = 634, t

688 out of 9734: movieID = 906, tmdbID = 13528
689 out of 9734: movieID = 907, tmdbID = 28288
690 out of 9734: movieID = 908, tmdbID = 213
691 out of 9734: movieID = 909, tmdbID = 284
692 out of 9734: movieID = 910, tmdbID = 239
693 out of 9734: movieID = 911, tmdbID = 4808
694 out of 9734: movieID = 912, tmdbID = 289
695 out of 9734: movieID = 913, tmdbID = 963
696 out of 9734: movieID = 914, tmdbID = 11113
697 out of 9734: movieID = 915, tmdbID = 6620
698 out of 9734: movieID = 916, tmdbID = 804
699 out of 9734: movieID = 917, tmdbID = 26531
700 out of 9734: movieID = 918, tmdbID = 909
701 out of 9734: movieID = 919, tmdbID = 630
702 out of 9734: movieID = 920, tmdbID = 770
703 out of 9734: movieID = 921, tmdbID = 31044
704 out of 9734: movieID = 922, tmdbID = 599
705 out of 9734: movieID = 923, tmdbID = 15
706 out of 9734: movieID = 924, tmdbID = 62
707 out of 9734: movieID = 926, tmdbID = 705
708 out of 9734: movieID = 927, tmdbID = 22490
709 out of 9734: movieID = 928, tmdbID = 22

...

9104 out of 9734: movieID = 144976, tmdbID = 294963
9105 out of 9734: movieID = 145080, tmdbID = 329020
9106 out of 9734: movieID = 145150, tmdbID = 298382
9107 out of 9734: movieID = 145283, tmdbID = 291343
9108 out of 9734: movieID = 145418, tmdbID = 294016
9109 out of 9734: movieID = 145491, tmdbID = 34729
9110 out of 9734: movieID = 145724, tmdbID = 74836
 -> production_companies = None
 -> production_countries = None
9111 out of 9734: movieID = 145745, tmdbID = 268788
9112 out of 9734: movieID = 145839, tmdbID = 321741
9113 out of 9734: movieID = 145935, tmdbID = 227973
9114 out of 9734: movieID = 145951, tmdbID = 36693
9115 out of 9734: movieID = 145994, tmdbID = 27940
9116 out of 9734: movieID = 146024, tmdbID = 45227
9117 out of 9734: movieID = 146028, tmdbID = 84254
9118 out of 9734: movieID = 146210, tmdbID = 288171
9119 out of 9734: movieID = 146244, tmdbID = 13986
9120 out of 9734: movieID = 146309, tmdbID = 315465
9121 out of 9734: movieID = 146656, tmdbID = 312221
9122 ou

9253 out of 9734: movieID = 156607, tmdbID = 290595
9254 out of 9734: movieID = 156609, tmdbID = 325133
9255 out of 9734: movieID = 156675, tmdbID = 29398
9256 out of 9734: movieID = 156706, tmdbID = 301608
9257 out of 9734: movieID = 156726, tmdbID = 376570
9258 out of 9734: movieID = 156781, tmdbID = 24784
9259 out of 9734: movieID = 156783, tmdbID = 49381
9260 out of 9734: movieID = 157108, tmdbID = 1659
9261 out of 9734: movieID = 157110, tmdbID = 1652
9262 out of 9734: movieID = 157122, tmdbID = 353326
9263 out of 9734: movieID = 157130, tmdbID = 366901
9264 out of 9734: movieID = 157172, tmdbID = 7237
9265 out of 9734: movieID = 157200, tmdbID = 303858
9266 out of 9734: movieID = 157270, tmdbID = 326423
9267 out of 9734: movieID = 157296, tmdbID = 127380
9268 out of 9734: movieID = 157312, tmdbID = 323676
9269 out of 9734: movieID = 157340, tmdbID = 153518
9270 out of 9734: movieID = 157369, tmdbID = 327536
9271 out of 9734: movieID = 157407, tmdbID = 332411
9272 out of 9734: mov

9407 out of 9734: movieID = 165529, tmdbID = 29136
9408 out of 9734: movieID = 165549, tmdbID = 334541
9409 out of 9734: movieID = 165551, tmdbID = 334543
9410 out of 9734: movieID = 165635, tmdbID = 419639
9411 out of 9734: movieID = 165639, tmdbID = 413279
9412 out of 9734: movieID = 165645, tmdbID = 338964
9413 out of 9734: movieID = 165671, tmdbID = 413361
9414 out of 9734: movieID = 165843, tmdbID = 393561
9415 out of 9734: movieID = 165947, tmdbID = 339396
9416 out of 9734: movieID = 165959, tmdbID = 33065
9417 out of 9734: movieID = 165969, tmdbID = 419546
9418 out of 9734: movieID = 166015, tmdbID = 393764
9419 out of 9734: movieID = 166024, tmdbID = 367412
9420 out of 9734: movieID = 166183, tmdbID = 77304
9421 out of 9734: movieID = 166203, tmdbID = 279229
9422 out of 9734: movieID = 166291, tmdbID = 378064
9423 out of 9734: movieID = 166461, tmdbID = 277834
9424 out of 9734: movieID = 166492, tmdbID = 384682
9425 out of 9734: movieID = 166526, tmdbID = 365942
9426 out of 973

9550 out of 9734: movieID = 173291, tmdbID = 339964
9551 out of 9734: movieID = 173307, tmdbID = 327253
9552 out of 9734: movieID = 173317, tmdbID = 390410
9553 out of 9734: movieID = 173351, tmdbID = 165095
9554 out of 9734: movieID = 173355, tmdbID = 193216
9555 out of 9734: movieID = 173535, tmdbID = 406403
 -> ERROR
9556 out of 9734: movieID = 173619, tmdbID = 30163
9557 out of 9734: movieID = 173751, tmdbID = 390587
 -> production_companies = None
 -> production_countries = None
9558 out of 9734: movieID = 173873, tmdbID = 26787
 -> ERROR
9559 out of 9734: movieID = 173925, tmdbID = 406990
9560 out of 9734: movieID = 173941, tmdbID = 341013
9561 out of 9734: movieID = 173963, tmdbID = 6079
9562 out of 9734: movieID = 174045, tmdbID = 336890
9563 out of 9734: movieID = 174053, tmdbID = 374430
 -> ERROR
9564 out of 9734: movieID = 174055, tmdbID = 374720
9565 out of 9734: movieID = 174141, tmdbID = 64847
9566 out of 9734: movieID = 174403, tmdbID = 461805
 -> ERROR
9567 out of 9734:

9692 out of 9734: movieID = 185029, tmdbID = 447332
9693 out of 9734: movieID = 185031, tmdbID = 399360
9694 out of 9734: movieID = 185033, tmdbID = 419831
9695 out of 9734: movieID = 185135, tmdbID = 500609
 -> ERROR
9696 out of 9734: movieID = 185435, tmdbID = 456750
9697 out of 9734: movieID = 185473, tmdbID = 437557
9698 out of 9734: movieID = 185585, tmdbID = 268896
9699 out of 9734: movieID = 186587, tmdbID = 427641
9700 out of 9734: movieID = 187031, tmdbID = 351286
9701 out of 9734: movieID = 187541, tmdbID = 260513
9702 out of 9734: movieID = 187593, tmdbID = 383498
9703 out of 9734: movieID = 187595, tmdbID = 348350
9704 out of 9734: movieID = 187717, tmdbID = 490003
9705 out of 9734: movieID = 188189, tmdbID = 424781
9706 out of 9734: movieID = 188301, tmdbID = 363088
9707 out of 9734: movieID = 188675, tmdbID = 483184
9708 out of 9734: movieID = 188751, tmdbID = 458423
9709 out of 9734: movieID = 188797, tmdbID = 455980
9710 out of 9734: movieID = 188833, tmdbID = 297725
97

#### 4. Explore the obtained dataframe

In [7]:
df_TMDB.shape

(9630, 13)

In [8]:
df_TMDB.head()

,tmdbID,movieID,adult,budget,original_language,popularity,poster_path,production_companies,production_countries,revenue,runtime,vote_average,vote_count
0,862,1,False,30000000,en,172.393,http://image.tmdb.org/t/p/original/uXDfjJbdP4i...,Pixar,United States of America,373554033,81,8.0,14716
0,8844,2,False,65000000,en,16.279,http://image.tmdb.org/t/p/original/6aGn2X51bah...,TriStar Pictures | Interscope Communications |...,United States of America,262797249,104,7.2,8589
0,15602,3,False,25000000,en,8.622,http://image.tmdb.org/t/p/original/1FSXpj5e8l4...,Lancaster Gate | Warner Bros. Pictures,United States of America,71500000,101,6.5,270
0,31357,4,False,16000000,en,7.878,http://image.tmdb.org/t/p/original/4wjGMwPsdlv...,20th Century Fox,United States of America,81452156,127,6.3,107
0,11862,5,False,0,en,11.100,http://image.tmdb.org/t/p/original/rj4LBtwQ0uG...,Touchstone Pictures | Sandollar Productions,United States of America,76594107,106,6.3,530


#### 5. Extract the dataframe to a .csv file

In [9]:
df_TMDB.to_csv(r'movies_apiTMDB.csv', index = False, header = True, encoding = 'utf-8-sig')